In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf

In [4]:
model = ResNet50(weights='imagenet')

In [5]:
## Labels ##
dic_labels = {}

files = open('dataset/labels_satellite/labels_classifier.txt')
lines = files.readlines()

for line in lines:
    name, label = line.split(' ')
    name = name.rstrip('.jpg')
    label = int(label.rstrip('\n'))
    dic_labels[name + '.png'] = label
print(dic_labels)

{'2MSI_20191226_S2-10.png': 0, '2MSI_20191226_S2-11.png': 0, '2MSI_20191226_S2-3.png': 0, '2MSI_20191226_S2-4.png': 0, '2MSI_20191226_S2-9.png': 1, '2MSI_20200110_S2.png': 1, '2MSI_20200130_S2-2.png': 1, '2MSI_20200130_S2-3.png': 1, '2MSI_20200130_S2-4.png': 1, '2MSI_20200130_S2-5.png': 1, '2MSI_20200130_S2.png': 1, '2MSI_20200224_S2-2.png': 1, '2MSI_20200224_S2.png': 1, '2MSI_20200315_S2.png': 1, '2MSI_20200320_S2-2.png': 1, '2MSI_20200320_S2-3.png': 1, '2MSI_20200320_S2.png': 1, 'MSI_20191214_S2.png': 1, 'MSI_20191216_S2-2.png': 0, 'MSI_20191216_S2.png': 0, 'MSI_20191219_S2.png': 1, 'MSI_20191221_S2-2.png': 0, 'MSI_20191221_S2.png': 1, 'MSI_20191224_S2.png': 1, 'MSI_20191226_S2-2.png': 1, 'MSI_20191226_S2.png': 1, 'MSI_20191227_S2.png': 1, 'MSI_20191228_S2.png': 1, 'MSI_20191229_S2.png': 1, 'MSI_20191231_S2.png': 1, 'MSI_20200108_S2.png': 1, 'MSI_20200126_S2.png': 1, 'MSI_20200129_S2.png': 0, 'MSI_20200130_S2.png': 1, 'MSI_20200202_S2.png': 1}


In [71]:
## Data ## 

# folder = 'dataset/images_satellite'

onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

print("Working with {0} images".format(len(onlyfiles)))
print("Image examples:")

# Print one of the examples
print(onlyfiles[0])
file_path = folder +"/" + onlyfiles[0]
print(file_path)
print(len(onlyfiles))

def preprocess(file_path):
    img = image.load_img(file_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    return x

def image_preprocess(folder):
    onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    target_height = 300
    target_width = 300
    inputs = np.zeros((len(onlyfiles), target_height, target_width, 3))
    
    for i in range(0, len(onlyfiles)):
        file_path = folder + "/" + onlyfiles[i]
        img = preprocess(file_path)
        inputs[i,:,:,:] = img
    return inputs,  len(onlyfiles)


inputs_pos, shape = image_preprocess('dataset/images_satellite')
inputs_neg, shape_neg = image_preprocess('dataset/images_unburnt')
print(inputs_pos.shape)
print(inputs_neg.shape)

Working with 27 images
Image examples:
2MSI_20200110_S2.png
dataset/images_satellite/2MSI_20200110_S2.png
27
(27, 300, 300, 3)
(8, 300, 300, 3)


In [51]:
# %%bash
# cd dataset
# mkdir images_unburnt_augmented

In [62]:
datagen = ImageDataGenerator(
        rotation_range=20,
        horizontal_flip=True,
        vertical_flip=True)

i = 0
for batch in datagen.flow(inputs_neg, batch_size=1,
                          save_to_dir='dataset/images_unburnt_augmented', save_format='jpg'):
    i += 1
    if i > 15:
        break  

In [63]:
inputs_pos, shape = image_preprocess('dataset/images_augmented')
inputs_neg, shape_neg = image_preprocess('dataset/images_unburnt_augmented')
print(inputs_pos.shape)
print(inputs_neg.shape)

inputs = np.concatenate((inputs_pos,inputs_neg), axis=0)
print(inputs.shape)
      
labels = np.zeros(inputs.shape[0])
labels[:shape] = 1
print(labels[:shape].shape)
print(labels)

(30, 300, 300, 3)
(46, 300, 300, 3)
(76, 300, 300, 3)
(30,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [66]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(inputs,labels,batch_size=50, epochs=5, verbose=1, shuffle = True)

Train on 76 samples
Epoch 1/5
76/76 [==============================] - 5s 72ms/sample - loss: 5.6763 - accuracy: 0.4605
Epoch 2/5
76/76 [==============================] - 0s 6ms/sample - loss: 0.3990 - accuracy: 0.7763
Epoch 3/5
76/76 [==============================] - 0s 6ms/sample - loss: 0.1165 - accuracy: 0.9605
Epoch 4/5
76/76 [==============================] - 0s 5ms/sample - loss: 0.0112 - accuracy: 1.0000
Epoch 5/5
76/76 [==============================] - 0s 5ms/sample - loss: 0.0026 - accuracy: 1.0000


In [72]:
img_test, _ = image_preprocess('dataset/test_images')
print(img_test.shape)
prediction = model.predict(img_test)
print(prediction)

(4, 300, 300, 3)
[[1.00000000e+00 6.91618336e-16]
 [1.00000000e+00 1.23658865e-30]
 [1.00000000e+00 4.12014626e-17]
 [9.97558951e-01 2.44100066e-03]]
